In [25]:
from nltk.tokenize import RegexpTokenizer
import cPickle as pickle
import string

%matplotlib inline
import numpy as np
import random
import time

import logging
from logging import info

In [29]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [2]:
SAMPLE_RATIO = 0.01

In [3]:
root_location = "/big/s/shalaby/"
exports_location = root_location + "exported_data/"

training_file = root_location + 'docs_output_training_validation_documents_' + str(SAMPLE_RATIO)
training_file = root_location + 'docs_output.json'

doc_classifications_map_file = exports_location + "doc_classification_map.pkl"
sections_file = exports_location + "sections.pkl"
classes_file = exports_location + "classes.pkl"
subclasses_file = exports_location + "subclasses.pkl"
classifications_output = exports_location + "classifications.pkl"
# training_docs_list_file = exports_location + "training_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
# validation_docs_list_file = exports_location + "validation_documents_" + str(SAMPLE_RATIO) + "_sample.pkl"
training_docs_list_file = exports_location + "training_docs_list.pkl"
validation_docs_list_file = exports_location + "validation_docs_list.pkl"

In [4]:
%%time
doc_classification_map = pickle.load(open(doc_classifications_map_file))
sections = pickle.load(open(sections_file))
classes = pickle.load(open(classes_file))
subclasses = pickle.load(open(subclasses_file))
training_docs_list = pickle.load(open(training_docs_list_file))
validation_docs_list = pickle.load(open(validation_docs_list_file))

CPU times: user 26.8 s, sys: 1.05 s, total: 27.8 s
Wall time: 28 s


In [5]:
len(training_docs_list)

1286325

In [7]:
len(validation_docs_list)

321473

In [9]:
def stemtokenizer(text):
    """ MAIN FUNCTION to get clean stems out of a text. A list of clean stems are returned """
    tokenizer = RegexpTokenizer(r'\s+', gaps=True)
    tokens = tokenizer.tokenize(text)
    stems = []  # result
    for token in tokens:
        stem = token.lower()
        stem = stem.strip(string.punctuation)
        if stem:
            if is_number(stem):
                stem = NUMBER_INDICATOR
            elif is_currency(stem):
                stem = CURRENCY_INDICATOR
            elif is_chemical(stem):
                stem = CHEMICAL_INDICATOR
            else:
                stem = stem.strip(string.punctuation)
            if stem and len(stem) >= MIN_SIZE:
                # extract uni-grams
                stems.append(stem)
    del tokens
    return stems

def is_number(str):
    """ Returns true if given string is a number (float or int)"""
    try:
        float(str.replace(",", ""))
        return True
    except ValueError:
        return False

def is_currency(str):
    return str[0] == "$"

def is_chemical(str):
    return str.count("-") > 3

## Get Separate Training and Validation Data Files

In [45]:
MIN_SIZE = 0
NUMBER_INDICATOR = "number_inidicator"
CURRENCY_INDICATOR = "currency_inidicator"
CHEMICAL_INDICATOR = "chemical_inidicator"


BATCH_SIZE = 10000
# TRAINING_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/training_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_data_preprocessed-" % str(SAMPLE_RATIO)
# VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX = "/mnt/data2/shalaby/validation_docs_sample_%s_docids_preprocessed-" % str(SAMPLE_RATIO)

TRAINING_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_data_preprocessed-"
TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/training_docs_docids_preprocessed-"
VALIDATION_PREPROCESSED_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_data_preprocessed-"
VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX = "/big/s/shalaby/preprocessed_data/validation_docs_docids_preprocessed-"

In [33]:
def write_batch(file_prefix, doc_files_prefix, batch_lines, doc_ids, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        with open(file_prefix + str(batch_start), 'w') as batch_file:
            for line in batch_lines:
                batch_file.write((u" ".join(line) + "\n").encode('utf-8'))
        pickle.dump(doc_ids, open(doc_files_prefix + str(batch_start), 'w'))


In [35]:
%%time
line_positions = dict()
with open(training_file) as f:
    
    i = 0
    line_positions[i] = f.tell()
    line = f.readline()
    while line:
        i+=1
        if not line.strip(): continue
        line_positions[i] = f.tell()
        line = f.readline()
    del line_positions[i]

CPU times: user 5min 7s, sys: 1min 12s, total: 6min 20s
Wall time: 13min 53s


Training

In [43]:
(divmod(len(line_positions), BATCH_SIZE)[0] + 1)

201

In [ ]:
range(150000, (divmod(len(line_positions), BATCH_SIZE)[0] + 1) * BATCH_SIZE, BATCH_SIZE )

In [ ]:
def multithreaded_batch_creation(start_index)

batch_index = 0
file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
doc_file_prefix = TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines, doc_ids = [], []
    start_time = time.time()
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in training_docs_list:
            token_lines.append(stemtokenizer(text))
            doc_ids.append(doc_id)
            if len(token_lines) % 100 == 0: info(len(token_lines))
            if len(token_lines) % BATCH_SIZE == 0:
                duration = time.time() - start_time
                info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
                start_time = time.time()
                write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines, doc_ids = [], []
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:d}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)

In [46]:
%%time

batch_index = 13
file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
doc_file_prefix = TRAINING_PREPROCESSED_DOCIDS_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines, doc_ids = [], []
    start_time = time.time()
    for index,line in enumerate(file_obj):
        if index < 130000: continue
        (doc_id, text) = eval(line)
        if doc_id in training_docs_list:
            token_lines.append(stemtokenizer(text))
            doc_ids.append(doc_id)
            if len(token_lines) % 1000 == 0: info(len(token_lines))
            if len(token_lines) % BATCH_SIZE == 0:
                duration = time.time() - start_time
                info("Finished batch of {:d} in {:.0f}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
                start_time = time.time()
                write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines, doc_ids = [], []
    duration = time.time() - start_time
    info("Finished batch of {:d} in {:d}m {:.0f}s".format(BATCH_SIZE, *divmod(duration, 60)))
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)

2017-01-02 01:25:14,018 : INFO : 1000
2017-01-02 01:28:01,824 : INFO : 2000
2017-01-02 01:31:05,860 : INFO : 3000
2017-01-02 01:34:10,195 : INFO : 4000
2017-01-02 01:37:09,988 : INFO : 5000
2017-01-02 01:40:11,487 : INFO : 6000
2017-01-02 01:43:12,095 : INFO : 7000
2017-01-02 01:46:12,877 : INFO : 8000
2017-01-02 01:49:08,692 : INFO : 9000
2017-01-02 01:52:01,441 : INFO : 10000
2017-01-02 01:52:01,443 : INFO : Finished batch of 10000 in 30m 3s


writing batch 130000


2017-01-02 01:55:17,627 : INFO : 1000
2017-01-02 01:58:21,699 : INFO : 2000
2017-01-02 02:01:28,690 : INFO : 3000
2017-01-02 02:04:28,969 : INFO : 4000
2017-01-02 02:07:29,446 : INFO : 5000
2017-01-02 02:10:30,742 : INFO : 6000
2017-01-02 02:13:35,014 : INFO : 7000
2017-01-02 02:16:34,744 : INFO : 8000
2017-01-02 02:19:35,035 : INFO : 9000
2017-01-02 02:22:30,112 : INFO : 10000
2017-01-02 02:22:30,113 : INFO : Finished batch of 10000 in 30m 29s


writing batch 140000


2017-01-02 02:25:43,022 : INFO : 1000
2017-01-02 02:28:37,431 : INFO : 2000
2017-01-02 02:31:30,380 : INFO : 3000
2017-01-02 02:34:28,167 : INFO : 4000
2017-01-02 02:37:32,448 : INFO : 5000
2017-01-02 02:40:32,475 : INFO : 6000
2017-01-02 02:43:27,390 : INFO : 7000
2017-01-02 02:46:22,148 : INFO : 8000
2017-01-02 02:49:26,378 : INFO : 9000
2017-01-02 02:52:24,805 : INFO : 10000
2017-01-02 02:52:24,806 : INFO : Finished batch of 10000 in 29m 55s


...

2017-01-03 00:38:56,022 : INFO : 1000
2017-01-03 00:41:52,106 : INFO : 2000
2017-01-03 00:44:53,635 : INFO : 3000
2017-01-03 00:47:54,551 : INFO : 4000
2017-01-03 00:50:52,444 : INFO : 5000
2017-01-03 00:53:48,344 : INFO : 6000
2017-01-03 00:56:37,635 : INFO : 7000
2017-01-03 00:59:34,075 : INFO : 8000
2017-01-03 01:02:33,885 : INFO : 9000
2017-01-03 01:05:25,933 : INFO : 10000
2017-01-03 01:05:25,934 : INFO : Finished batch of 10000 in 29m 55s


writing batch 590000


2017-01-03 01:08:44,106 : INFO : 1000
2017-01-03 01:11:52,003 : INFO : 2000
2017-01-03 01:14:50,026 : INFO : 3000
2017-01-03 01:17:51,948 : INFO : 4000
2017-01-03 01:20:57,639 : INFO : 5000
2017-01-03 01:23:50,747 : INFO : 6000
2017-01-03 01:27:03,418 : INFO : 7000
2017-01-03 01:30:12,214 : INFO : 8000
2017-01-03 01:33:24,545 : INFO : 9000
2017-01-03 01:36:28,362 : INFO : 10000
2017-01-03 01:36:28,363 : INFO : Finished batch of 10000 in 31m 2s


writing batch 600000


KeyboardInterrupt: 

Validation

In [8]:
%%time

batch_index = 0
file_prefix = VALIDATION_PREPROCESSED_FILES_PREFIX
doc_file_prefix = VALIDATION_PREPROCESSED_DOCIDS_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines, doc_ids = [], []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in validation_docs_list:
            token_lines.append(stemtokenizer(text))
            doc_ids.append(doc_id)
            if len(token_lines) % BATCH_SIZE == 0:
                %time write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines, doc_ids = [], []
    write_batch(file_prefix, doc_file_prefix, token_lines, doc_ids, batch_index * BATCH_SIZE)

writing batch 0
CPU times: user 42.5 s, sys: 832 ms, total: 43.4 s
Wall time: 43.5 s


In [34]:
%%time
line_tokens = []
with open(TRAINING_PREPROCESSED_FILES_PREFIX + str(0)) as preproc_file:
    line_lengths = []
    for line in preproc_file:
        line_lengths.append(len(line))
        line_tokens.append(line.split(" "))

CPU times: user 1.53 s, sys: 856 ms, total: 2.38 s
Wall time: 2.39 s


### Old Method

we used to write the doc id and the tokens as tuples, then do an eval on them in reading time, but this turned out to be very slow

Training

In [ ]:
%%time
def write_batch(file_prefix, batch_lines, batch_start):
    if len(batch_lines):
        print "writing batch %d" % batch_start
        %time pickle.dump(batch_lines, open(file_prefix + str(batch_start), 'w'))
#         with open(file_prefix + str(batch_start), 'w') as batch_file:
#             for line in batch_lines:
#                 batch_file.write(str(line) + "\n")

batch_index = 0
file_prefix = TRAINING_PREPROCESSED_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines = []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in training_docs_list:
            token_lines.append((doc_id, stemtokenizer(text)))
            if len(token_lines) % BATCH_SIZE == 0:
                write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines = []
    write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)

Validation

In [8]:
batch_index = 0
file_prefix = VALIDATION_PREPROCESSED_FILES_PREFIX
with open(training_file) as file_obj:
    token_lines = []
    for line in file_obj:
        (doc_id, text) = eval(line)
        if doc_id in validation_docs_list:
            token_lines.append((doc_id, stemtokenizer(text)))
            if len(token_lines) % BATCH_SIZE == 0:
                write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)
                batch_index += 1
                token_lines = []
    write_batch(file_prefix, token_lines, batch_index * BATCH_SIZE)

writing batch 0
writing batch 10000


In [22]:
%%time
line_tokens = []
with open(TRAINING_PREPROCESSED_FILES_PREFIX + str(0)) as preproc_file:
    for line in preproc_file:
        line_tokens.append(line.split(" "))

CPU times: user 5 s, sys: 1.2 s, total: 6.19 s
Wall time: 6.18 s


In [24]:
line_tokens[0][:10]

['technical',
 'field',
 'the',
 'present',
 'invention',
 'generally',
 'relates',
 'to',
 'wireless',
 'communications']